# P2I7 - Le pendule double

## Comment modéliser numériquement le comportement non linéaire du pendule double ?


**Projet mené par :** Lomenech Nathan, Gagou Mattéo, Roche Nathan, Mauclaire Alexis

<div class="alert alert-danger">
<b>Important:</b> 
    
Pour avoir l'entièreté du code : 
   
</div>

<img src="images/pendule_double.gif" alt="Texte alternatif" style="width:400px;height:400px;">

## Table des matières : 

<u>1. Introduction </u>

    1. Qu’est-ce qu’un  pendule double ?
    2. Objectifs de notre projet
    3. Etude de la problématique


<u>2. Présentation du système</u>

    1. Schema et nomenclature
    2. Contexte linéaire et non linéaire
    4. Solution analytique au contexte linéaire
    5. Energie totale du système

<u>3. Méthodes de résolutions utilisées : </u>
    
    1. Newmark/Newton-raphson
    2. Solve ivp
    
<u>4. Résolution des différents modèles: </u>

    1. Comparaison des résultats au contexte linéaire 
    2. Comparaison des résultats au contexte non linéaire 
    
<u>5. Comparaison linéaire / non linéaire :</u>

    1. Détermination de l'angle critique
    
<u>6. Analyse du comportement : </u>

    1. Les outils à disposition
    2. Determination comportement
    3. Aspect chaotique
    4. Critique des résultats et améliorations

# Introduction

## 1.1 Qu'es ce qu'un pendule double ?

Mettre un schema explicatif

**Pendule Simple** : Le pendule simple est un système périodique qui se compose d'une masse suspendue à un fil inextensible et sans masse. Il oscille autour d'un point d'équilibre en suivant un mouvement harmonique simple, avec une période de oscillation dépendant de la longueur du fil et de l'accélération gravitationnelle. Les positions du pendule se répètent de manière régulière et prédictible.

**Pendule Double** : Le pendule double est formé de deux pendules simples attachés l'un à l'extrémité de l'autre. Ce système est chaotique et non linéaire, ce qui signifie que son comportement est extrêmement sensible aux conditions initiales. Contrairement au pendule simple, les mouvements du pendule double ne peuvent pas être prédits de manière précise sur de longues périodes, en raison de la complexité de ses interactions internes.

**Comparaison** : Le pendule simple et le pendule double illustrent deux types de systèmes dynamiques opposés. Le pendule simple représente un système périodique et linéaire, avec un mouvement régulier et prévisible. En revanche, le pendule double est un exemple de système chaotique et non linéaire, caractérisé par une sensibilité accrue aux conditions initiales et un comportement imprévisible à long terme. Cette comparaison met en évidence la transition entre la simplicité des systèmes périodiques et la complexité des systèmes chaotiques.


## 1.2 Objectifs du projet

Dans ce projet, nous avons fréquemment comparé deux hypothèses, que vous entendrez souvent au cours de l’exposé :

1. **Intégration des équations du mouvement du pendule double avec l’approximation des petits angles** : Dans ce contexte, l’oscillateur reste linéaire (amplitude de mouvement faible).
2. **Intégration des équations du mouvement non linéaire du pendule double**.

Les principales étapes de notre analyse incluent :

- **Analyse du comportement du pendule en fonction des conditions initiales**.
- **Détermination de l’angle critique à partir duquel le mouvement devient chaotique**.
- **Étude de la stabilité du pendule à l'aide de l’exposant de Lyapunov**.

## 1.3  Etude de la problématique

Le pendule double est un système fascinant à modéliser en raison de sa dynamique complexe et non linéaire, illustrant le chaos déterministe où de petites variations des conditions initiales entraînent des comportements radicalement différents. Ce projet vise à répondre à la question suivante : Comment modéliser numériquement le comportement non linéaire du pendule double

# 2. Présentation du système

## 2.1 Schéma et nomenclature

#### On utilise le schéma suivant pour la nomenclature des angles et longeurs :
<img src="images/schema.png" alt="Texte alternatif" style="width:300px;height:300px;">

#### Les différentes variables aux petits angles: 
 - solution_ana_pa : **solution analytique aux petits angles**, array 2D avec [$\theta_1$,$\theta_2$, $\dot{\theta_1}$,$\dot{\theta_1}$]
 - solution_ivp_pa : **solution obtenu avec solve.ivp** array 2D avec [$\theta_1$, $\theta_2$,$\dot\theta_1$,$\dot{\theta_2}$]
 - solution_newmark_pa :  **solution obtenu avec l'algorithme de Newmark** array 2D avec [$\theta_1$,$\theta_2$, $\dot{\theta_1}$,$\dot{\theta_1}$]
 - x0 = [$\theta_{10}$,$\theta_{20}$,$\dot\theta_{10}$, $\dot\theta_{20}$]

#### Les différentes variables aux grands angles: 
 - solution_ivp_ga : **solution obtenu avec solve.ivp** array 2D avec [$\theta_1$, $\dot{\theta_1}$,$\theta_2$,$\dot{\theta_1}$]
 - solution_newmark_ga :  **solution obtenu avec l'algorithme de Newmark** array 2D avec [$\theta_1$,$\theta_2$, $\dot{\theta_1}$,$\dot{\theta_1}$]
- x0 = [$\theta_{10}$,$\theta_{20}$,$\dot\theta_{10}$, $\dot\theta_{20}$]


 **Nomenclature des fonctions/variables** :
 - _pa = petits angles -> contexte non linéaire
 - _ga = grands angles -> contexte linéaire


## 2.2 Contexte non linéaire

#### Equations différentielles dans le contexte non linéaire:

Dans un premier temps, en adoptant une approche newtonienne, nous parvenons à formuler un système d'équations différentielles non linéaires décrivant l'évolution du pendule. Ce système, étant particulièrement complexe, ne peut être résolu de manière analytique, c'est-à-dire qu'aucune solution sous forme littérale n'est disponible pour les angles $ \theta_1 $ et $ \theta_2 $ en fonction du temps.

$\hspace{3cm} 
\begin{cases} 
(m_1 + m_2)l_1\ddot\theta_1 + m_2 l_2 \cos(\theta_1 - \theta_2)\ddot\theta_2 + m_2 l_2 \sin(\theta_1 - \theta_2)\dot\theta_2^2 + (m_1 + m_2)g \sin\theta_1 = 0
\\ 
m_2 l_2 \ddot\theta_2 + m_2 l_1 \cos(\theta_1 - \theta_2)\ddot\theta_1 - m_2 l_1 \sin(\theta_1 - \theta_2)\dot\theta_1^2 + m_2 g \sin\theta_2 = 0
\end{cases}$

## 2.3 Linéarisation aux petits angles

En procédant à l'approximation des petits angles, nous pouvons obtenir un système d'équations différentielles linéaires pour lesquelles une solution analytique est possible. Nous utiliserons cette solution analytique pour calibrer nos modèles dans les contextes linéaires ainsi que dans les contextes non linéaires avec de petits angles initiaux.

$\hspace{3cm} 
\begin{cases} 
l_1 \ddot{\theta_1} + (1 + \mu)g \theta_1 -\mu g \theta_2 = 0
\\ 
l_2 \ddot{\theta_2} - (1+\mu)g \theta_1 + (1+\mu)g \theta_2 = 0
\end{cases}$

avec $\mu = \frac{m_2}{m_1}$


## 2.4 Solution analytique aux petits angles

On a le système suivant :

$\hspace{3cm} 
\begin{cases} 
(m_1 + m_2)l_1^2 \ddot{\theta}_1 + m_2l_1l_2\ddot{\theta}_2 + (m_1 + m_2)gl_1\theta_1 = 0
\\ 
m_2l_1l_2\ddot{\theta}_2 + m_2l_2^2\ddot{\theta}_2 + m_2gl_2\theta_2 = 0
\end{cases}$


Nous allons utilisé la version suivante : 

$\hspace{3cm} 
\begin{cases} 
\ddot{\theta}_1 = \frac{\mu g\theta_2 - (1+\mu)g\theta_1}{l_1}
\\ 
\ddot{\theta}_2 = \frac{(1+\mu)g \theta_1 - (1+\mu)g\theta_2}{l_2}
\end{cases}$

Avec la solution analytique exacte : 

$\hspace{3cm} 
\begin{cases} 
\theta_1^{exact}(t) = C_1 \cos(\omega_1t + \phi_1) + C_2 \cos(\omega_2t + \phi_2)
\\
\theta_2^{exact}(t) = C_1 A_1 \cos(\omega_1t + \phi_1)+C_2 A_2 \cos(\omega_2 t + \phi_2)
\end{cases}$

avec : 

$$ \omega_{1,2}^2 = \frac{g(1+ \mu)(l_1+l_2) \pm g\sqrt{(1+\mu)^2(l_1+l_2)^2 - 4(1+\mu)l_1l_2}}{2l_1l_2} $$


<div class="alert alert-danger">
<b>Important:</b> 
    
Nous distinguons ainsi deux contextes :
- Le contexte linéaire, dans lequel nous nous limitons à des angles initiaux faibles et utilisons les équations linéaires associées.
- Le contexte non linéaire, qui s'applique à l'ensemble des conditions initiales possibles, avec les équations différentielles correspondantes.

</div>

## 2.5 Energie totale du système 

$\hspace{3cm} 
\begin{cases} 
E_{c1} = -m_1 g l_1 \cos\theta_1
\\ 
E_{c2} = \frac{1}{2}m_2(l_1^2\dot\theta_1^2 +l_2^2 \dot\theta_2^2 + 2l_1 l_2 \cos(\theta_1 - \theta_2)\dot\theta_1 \dot\theta_2 )
\\
E_{p1} = -m_1 g l_1 \cos\theta_1
\\
E_{p2} = -m_2 g (l_1 \cos\theta_1 + l_2\cos\theta_2)
\\
E_{tot} = E_{c1} + E_{c2} + E_{p1} + E_{p2}
\end{cases}$

# 3. Méthodes de résolutions utilisées : 

## 3.1 Newmark/Newton-Raphson

### Introduction

La méthode de `Newmark` est une technique numérique utilisée pour résoudre les équations différentielles ordinaires (EDO), souvent rencontrées dans la modélisation des systèmes dynamiques comme le pendule double. Elle repose sur une approche itérative qui prédit d'abord les positions, vitesses et accélérations angulaires à un pas de temps ultérieur, puis affine ces prédictions pour minimiser les erreurs.

#### Nomenclature
- X = arrray numpy 2D des positions angulaires en fonction du temps : X = [$\theta_1$,$\theta_2$]
- Xp = array numpy 2D des vitesses angulaires en fonction du temps : Xp =[$\dot\theta_1$,$\dot\theta_2$]
- Xp = array numpy 2D des accelerations angulaires en fonction du temps : Xpp =[$\ddot\theta_1$,$\ddot\theta_2$]
- precNR : précision de résolution du système d'équations différentielles à chaque pas de temps
- Pour les **Jacobiennes** :
    - Jx : matrice des dérivées partielles en fonction de $\theta_1$,$\theta_2$. Donc en fonction des positions angulaires
    - Jxp : matrice des dérivées partielles en fonction de $\dot\theta_1$,$\dot\theta_2$. Donc en fonction des vitesses angulaires
    - Jxpp = C : matrice des dérivées partielles en fonction de $\ddot\theta_1$,$\ddot\theta_2$. Donc en fonction des accélérations angulaires
    
### Fonctionement

#### Initialisation des variables en fonction des CI

Avant de rentrer dans la boucle, on initialise les variables de position et d'accélération dans les arrays \( X \) et \( Xp \) en fonction des conditions initiales renseignées dans l'appel de la fonction.

#### Résolution de l'accélération à \( t=0 \)

Afin de pouvoir continuer la résolution du modèle, il faut d'abord caractériser totalement le système à \( t=0 \). Pour cela, on calcule le vecteur accélération \( Xpp \) à \( t=0 \) en initialisant les matrices A et B.

#### Boucle temporelle

##### Prédiction des positions/vitesses/accélérations angulaires

On tente de prédire les positions, vitesses et accélérations angulaires au pas de temps \( t+1 \) pour avoir une base de départ pour affiner ces valeurs par la suite :


\begin{cases} 
X_{t+1} = X_t + Xp_t dt + \frac{1}{2}Xpp_t dt^2
\\ 
Xp_{t+1} = Xp_t + Xpp_t dt
\\
Xpp_{t+1} = Xpp_t
\end{cases}


##### Calcul du résidu de la prédiction

Pour estimer l'erreur faite dans la première prédiction, on calcule le résidu associé aux valeurs prédites. On réinjecte ces valeurs dans le système d'équations différentielles et on calcule le résidu associé.

##### Boucle afin de satisfaire la condition sur le résidu

On affine la prédiction pour satisfaire la condition de précision définie précédemment (precNR), tout en évitant de boucler indéfiniment. Pour cela, on itère la méthode de Newton-Raphson adaptée aux équations différentielles, pondérant le résultat ajouté aux valeurs précédentes par le résidu précédent.

Afin d'adapter la méthode de Newton-Raphson aux équations différentielles, on calcule des jacobiennes Jx, Jxp, Jxpp (voir nomenclature) que l'on pondère en fonction de \( dt \) pour obtenir une jacobienne du système différentiel. On met ensuite à jour les valeurs de positions, vitesses et accélérations angulaires, puis on recalcule le résidu avec ces nouvelles valeurs.

Ce processus itère pour chaque pas de temps jusqu'à ce que la condition de précision precNR soit satisfaite.

### Avantages de Newmark

- **Adaptabilité** : La méthode de Newmark est adaptable à une variété de systèmes dynamiques, y compris le pendule double, grâce à sa capacité à prédire et affiner les valeurs de positions, vitesses et accélérations.
- **Précision** : En itérant pour minimiser le résidu, la méthode de Newmark peut fournir des solutions précises aux équations différentielles, même pour des systèmes complexes.
- **Polyvalence** : Cette méthode peut être appliquée à des problèmes de dynamique non linéaire, où d'autres méthodes peuvent rencontrer des difficultés.

### Inconvénients de Newmark

- **Complexité** : La mise en œuvre de la méthode de Newmark peut être complexe, en particulier pour des systèmes avec des conditions initiales complexes ou des non-linéarités importantes.
- **Coût calculatoire** : En raison de son approche itérative, la méthode de Newmark peut être plus coûteuse en termes de ressources de calcul par rapport à d'autres techniques pour des problèmes de grande taille ou à haute dimension.

En résumé, la méthode de Newmark offre une approche puissante pour résoudre numériquement les équations différentielles ordinaires, mais elle nécessite une compréhension approfondie de la théorie sous-jacente et peut être plus coûteuse en termes de calcul dans certains cas.


## 3.2 Solve ivp

La fonction `solve_ivp` est un outil puissant utilisé dans les bibliothèques scientifiques pour résoudre numériquement des équations différentielles ordinaires (EDO), en mettant l'accent sur la méthode de résolution. Cette fonction est particulièrement utile pour traiter des problèmes de Cauchy, une classe spécifique de problèmes de valeurs initiales pour les EDO.

### Problème de Cauchy

Un problème de Cauchy est une équation différentielle ordinaire accompagnée de conditions initiales spécifiées à un instant donné. Il s'écrit généralement sous la forme :

$$\frac{dy}{dt} = f(t, y), \quad y(t_0) = y_0$$

où :
- $t$ est la variable indépendante (souvent le temps),
- $y$ est la variable dépendante (souvent une fonction de $t$),
- $f(t, y)$ est une fonction donnée qui décrit le taux de changement de $y$ par rapport à $t$,
- $t_0$ est l'instant initial,
- $y_0$ est la valeur initiale de $y$ à l'instant $t_0$.

Le but est de trouver la fonction $y(t)$ qui satisfait à la fois l'équation différentielle et les conditions initiales.

### Méthode de résolution des EDO

La résolution des EDO est effectuée en subdivisant l'intervalle de temps spécifié en plusieurs pas discrets. À chaque pas, la fonction `solve_ivp` utilise une méthode numérique pour estimer la valeur de la solution à ce point. Dans notre cas, nous avons utilisé la méthode **Runge-Kutta (RK)**. En effet, cette méthode est largement utilisée pour sa simplicité et sa robustesse. Elle utilise une combinaison pondérée de pentes calculées à différents points dans l'intervalle de temps pour estimer la valeur de la solution à l'étape suivante.

### Avantages de la méthode de résolution de solve_ivp

- **Adaptabilité** : La fonction `solve_ivp` sélectionne automatiquement la méthode de résolution la plus appropriée en fonction des propriétés du problème, tels que la raideur et la régularité de la solution.
- **Efficacité** : Les méthodes utilisées sont conçues pour être efficaces en termes de temps de calcul, même pour des problèmes complexes.
- **Polyvalence** : La fonction `solve_ivp` peut traiter une grande variété de problèmes, des simples aux plus complexes, en offrant une solution fiable pour une large gamme de situations.

### Inconvénients de la méthode de résolution de solve_ivp

- **Sensibilité aux conditions initiales** : Certains types de méthodes peuvent être plus sensibles aux conditions initiales que d'autres, nécessitant un réglage fin pour garantir la stabilité de la solution.
- **"Boite noire"** : On ne controle pas les algorithmes utilisés, ce qui rend compliqué de modifier la fonction pour des problèmes complexes comme le pendule double

En conclusion, la fonction `solve_ivp` se présente comme une boîte noire puissante pour la résolution numérique des équations différentielles ordinaires. Grâce à sa méthode de résolution automatique, elle offre une solution fiable et précise pour une grande variété de problèmes, en adaptant dynamiquement les méthodes en fonction des caractéristiques du système. Cependant, il est important de noter que son fonctionnement interne reste opaque, ce qui signifie que nous ne contrôlons pas directement les choix algorithmiques effectués. Ainsi, bien que cette fonctionnalité soit extrêmement utile, il est crucial de comprendre les spécificités de chaque méthode et d'ajuster les paramètres en conséquence pour garantir la stabilité et la précision des résultats.

#  4. Résolution des différents modèles

<div class="alert alert-danger">
<b>Important:</b> 
    
Pour la suite du rapport, toutes les simualtions ont été effectués avec variables suivantes : 
    <ul>
    <li>$l_1$ = 3m</li>
    <li>$l_2$ = 2m</li>
    <li>$m_1$ = 2kg</li>
    <li>$m_2$ = 3kg</li> 
    <li>$dt$ = 0.005s</li>
    <li>$\dot\theta_{10}$ = 0 rd/s</li>
    <li>$\dot\theta_{20}$ = 0 rd/s</li>
</ul>

   
</div>

## 3.1 Comparaison des résultats au contexte linéaire

Au contexte linéaire, on dispose d'une solution analytique, ce qui nous permet de comparer nos méthodes d'intégrations numériques à une référence afin d'en améliorer l'efficacité. Après avoir paramétré les méthodes de Newmark/Newton-Raphson et solve_ivp, on arrive aux résultats suivants :

### Illustrations graphiques : 
On a ici $\theta_{10} = \theta_{20}$ = 5°

#### Variations des angles au cours du temps
<img src="images/angles pa.png" alt="Texte alternatif">

#### Ecart relatif à la solution analytique
<img src="images/ecart pa.png" alt="Texte alternatif">

#### Ecart relatif à l'énergie initiale
<img src="images/energie pa.png" alt="Texte alternatif">



### Analyse : 


## 3.2 Comparaison des résultats au contexte non linéaire

Au contraire du contexte linéaire, on ne dispose pas d'une solution analytiqueau contexte non linéire.

### Illustrations graphiques : 
On a ici $\theta_{10} = \theta_{20}$ = 120°

#### Variations des angles au cours du temps
<img src="images/angles ga.png" alt="Texte alternatif">
A interpréter modulo $2\pi$

#### Ecart relatif à la solution analytique
<img src="images/ecart ga.png" alt="Texte alternatif">

#### Ecart relatif à l'énergie initiale
<img src="images/energie ga.png" alt="Texte alternatif">



### Analyse :
On utilisera newmark pour la suite

# 5. Comparaison linéaire / non linéaire

## 5.1 Détermination de l'angle critique

Maintenant que notre modèle a été validé, notre prochaine étape consiste à comparer les comportements linéaires et non linéaires afin de déterminer la plage dans laquelle l'approximation des petits angles est valide.

Pour ce faire, nous avons généré une heatmap représentant l'écart entre les deux modèles en fonction des conditions initiales $ \theta_1 $ et $ \theta_2 $, où $ \theta_1 $ est en ordonnée et $ \theta_2 $ en abscisse. Une observation rapide révèle que lorsque les angles sont de faible amplitude et de même signe, l'écart entre les deux contextes reste faible. Cependant, lorsque les angles sont de signes opposés, l'écart entre les deux devient significatif.

<img src="resultats/comparaison newmark pa-ga max.png" alt="Texte alternatif" style="width:700px;height:700px;">



### Illustrations graphiques : 


<div style="display: flex;">
    <div style="flex: 1; padding-right: 20px; max-width: 50%;">
        <h2>Domaine non linéaire</h2>
        <p>Avec les conditions initiales suivantes qui sont censées être dans le domaine non linéaire (violet sur la heatmap), on observe que les trajectoires des modèles linéaires et non linéaires divergent, validant les calculs précédents.</p>
        <p>Conditions initiales: $\theta_{10}$ = -60° et $\theta_{20}$ = 70°</p>
        <video controls src="animations/paga ga.mp4" style="width: 100%; max-width: 400px;" />
    </div>
    <div style="flex: 1; padding-left: 20px; max-width: 50%;">
        <h2>Domaine linéaire</h2>
        <p>Avec les conditions initiales suivantes qui sont censées être dans le domaine linéaire (bleu foncé sur la heatmap), on observe que les trajectoires des modèles linéaires et non linéaires se confondent, validant les calculs précédents.</p>
        <p>Conditions initiales: $\theta_{10}$ = 15° et $\theta_{20}$ = 15°</p>
        <video controls src="animations/paga pa.mp4" style="width: 100%; max-width: 400px;" />
    </div>
</div>


### Conclusion
Grâce à ces résultats, nous sommes en mesure de définir une plage valide où l'approximation des petits angles, et donc le contexte linéaire, est justifiable. De manière intéressante, nous observons que cette approximation peut être étendue à des angles plus importants lorsque ces derniers sont de même signe. En revanche, pour les angles de signes opposés, cette extension est moins applicable. Pour couvrir tous les cas possibles, nous utiliserons par la suite la méthode de **Newmark adaptée au contexte non linéaire**, assurant ainsi une analyse exhaustive du comportement du système du pendule double.

# 6. Analyse du comportement

## 6.1 Les outils à disposition

<div style="display: flex;">
    <div style="flex: 1; padding-right: 20px; max-width: 33%;">
        <h2>Portrait de phase</h2>
        <p>Un portrait de phase est une représentation graphique des relations entre les variables dynamiques d'un système au fil du temps. Pour un pendule double, il montre les positions et les vitesses angulaires du pendule pendant son mouvement, chaque point représentant un état du système. Cette visualisation est utile pour comprendre le comportement du pendule car elle permet de voir ses trajectoires dans l'espace des phases. En identifiant les régions correspondant à des mouvements périodiques, chaotiques ou stables, ainsi que les points fixes ou les cycles limites, on obtient des informations clés sur la dynamique du système.
            
**Exemple portrait de phase :** </p>
        <img src="images/exemple portrait de phase.png" width="300"/>
    </div>
    <div style="flex: 1; padding-right: 20px; padding-left: 20px; max-width: 33%;">
        <h2>Section de Poincaré</h2>
        <p>La section de Poincaré est une méthode d'analyse essentielle pour étudier le comportement dynamique des systèmes non linéaires tels que le pendule double. Elle consiste à observer les intersections d'une trajectoire dynamique avec un plan de coupe spécifique dans l'espace des phases du système. Cette approche réduit la complexité de l'espace des phases, permettant ainsi une visualisation plus claire des comportements périodiques, des bifurcations et même du chaos. Pour le pendule double, l'analyse de la section de Poincaré permet d'identifier des structures dynamiques complexes et d'explorer les conditions initiales et les paramètres du système conduisant à des phénomènes intéressants.
            
**Exemple section de Poincaré :** </p>
        <img src="images/exemple section poincaré.png" width="400"/>
    </div>
    <div style="flex: 1; padding-left: 20px; max-width: 33%;">
        <h2>Diagramme de bifurcation</h2>
        <p>Le diagramme de bifurcation est un outil clé pour comprendre les changements dans le comportement d'un système dynamique en fonction d'un paramètre de contrôle. Pour le pendule double, il montre comment les états stables du système varient avec des paramètres comme la longueur des bras ou la masse des masses. En observant ce diagramme, on peut repérer les points où le système subit des changements qualitatifs dans son mouvement, comme l'apparition de nouvelles trajectoires ou le passage au chaos. Comprendre ce diagramme permet d'analyser les différents comportements du pendule double en fonction de ses paramètres, offrant ainsi un aperçu précieux de sa dynamique..
            
**Exemple diagramme de bifurcation :** </p>
        <img src="images/exemple diagramme de bifurcation.png" width="300"/>
    </div>
</div>


## 6.2 Détermination du comportement

### Comportement en fonction des conditions initiales :

<div style="display: flex;">
    <div style="flex: 1; padding-right: 20px; max-width: 50%;">
        <h2>Petits angles</h2>
        <p>Conditions initiales: $\theta_{10}$ = 30° et $\theta_{20}$ = 30°</p>
        <p><b>Portrait de phase</b></p>
        <img src="images/portrait de phase pa1.png" width="500"/>
        <img src="images/portrait de phase pa2.png" width="500"/>
        <p><b>Sections de Poincaré</b></p>
        <img src="images/poincaré pa1.png" width="500"/>
        <img src="images/poincaré pa2.png" width="500"/>
        <p> <b>Analyse :</b> 
            L'observation des portraits de phases et des sections de Poincaré des deux angles révèle la présence de trajectoires présentant une régularité et une répétitivité caractéristiques, suggérant un mouvement périodique. Cependant, une analyse plus approfondie révèle que la forme de ces trajectoires n'est pas parfaitement circulaire, indiquant un comportement pseudo-périodique. </p>
    </div>
    <div style="flex: 1; padding-left: 20px; max-width: 50%;">
        <h2>Grands angles</h2>
        <p>Conditions initiales: $\theta_{10}$ = 120° et $\theta_{20}$ = 120°</p>
        <p><b>Portrait de phase</b></p>
        <img src="images/portrait de phase ga1.png" width="500"/>
        <img src="images/portrait de phase ga2.png" width="500"/>
        <p><b>Sections de Poincaré</b></p>
        <img src="images/poincaré ga1.png" width="500"/>
        <img src="images/poincaré ga2.png" width="500"/>
        <p><b>Analyse :</b> 
        L'analyse des portraits de phases et des sections de Poincaré des deux angles révèle un schéma de trajectoires irrégulières et imprévisibles au fil du temps, témoignant d'un comportement chaotique. Contrairement au mouvement périodique, les trajectoires dans ces représentations ne présentent pas de structure régulière et semblent sensiblement différentes d'une itération à l'autre. Cette observation suggère une dynamique complexe et non déterministe, caractéristique du comportement chaotique.</p>
    </div>
</div>


### Diagramme de biffurcation

On pose ici : $\theta_{10} = \theta_{20}$
<img src="images/bifurcation avec domaine.png"/>

#### Analyse du diagramme 

À des valeurs plus faibles de l'angle initial, le système du pendule double semble exhiber un comportement pseudo-périodique, avec des solutions quasiment stables. Cependant, à mesure que l'angle initial augmente, le système entre dans un régime de bifurcation, où plusieurs états ou solutions deviennent possibles. Cette transition vers des comportements dynamiques plus complexes est particulièrement notable lorsque l'angle initial approche les 60 degrés, considéré comme un angle critique pour ce pendule double. Au-delà de cet angle critique, le système montre des signes de chaos, avec des trajectoires de plus en plus irrégulières et imprévisibles, indiquant une instabilité croissante du mouvement. Ainsi, l'analyse du comportement du pendule double révèle une transition significative vers des régimes pseudo-périodiques et chaotiques à mesure que l'angle initial dépasse les 60 degrés.


<div style="display: flex;">
    <div style="flex: 1; padding-right: 20px; max-width: 50%;">
        <h2>Domaine pseudo périodique</h2>
        <p>L'analyse du pendule révèle une tendance où les trajectoires oscillatoires quasi-récurrentes se reproduisent presque identiquement à chaque pseudo-période. Ceci témoigne d'un comportement pseudo-périodique observé dans le mouvement du pendule.</p>
        <p>Conditions initiales: $\theta_{10} = \theta_{20}$ = 50°</p>
        <video controls src="animations/50.mp4" style="width: 100%; max-width: 400px;" />
    </div>
    <div style="flex: 1; padding-left: 20px; max-width: 50%;">
        <h2>Domaine chaotique</h2>
        <p>On constate que le pendule ne répète pas de manière identique ses trajectoires à chaque pseudo-période, indiquant ainsi un comportement chaotique.</p>
        <p>Conditions initiales: $\theta_{10} = \theta_{20}$ = 140°</p>
        <video controls src="animations/140.mp4" style="width: 100%; max-width: 400px;" />
    </div>
</div>


#### Conclusion : 
L'analyse des portraits de phase et des sections de Poincaré révèle une transition du comportement du pendule double en fonction de l'angle initial. À de faibles angles, le système exhibe un comportement pseudo-périodique, avec des solutions presque stables. Cependant, à mesure que l'angle initial augmente, le système entre dans un régime de bifurcation, menant à des comportements chaotiques caractérisés par des trajectoires irrégulières et imprévisibles. Ainsi, cette analyse met en évidence une transition significative du comportement du pendule double, de pseudo-périodique à chaotique, en fonction de l'angle initial.

## 6.3 Aspect chaotique

### Qu'es ce que l'exposant de Lyapunov ?


<div style="display: flex;">
    <div style="flex: 1; padding-right: 20px; max-width: 50%;">
    <p>L'exposant de Lyapunov est une mesure quantitative de la sensibilité aux conditions initiales d'un système dynamique. Il caractérise l'évolution de la divergence entre deux trajectoires initialement très proches dans l'espace des phases. Dans notre cas, où l'écart entre les deux angles initiaux est de 1%, l'exposant de Lyapunov nous permet de quantifier le degré de chaos dans le système. Plus précisément, il indique à quel point le comportement du système dépend de ses conditions initiales.</p>
    <p>Pour calculer l'exposant de Lyapunov, on suit généralement ces étapes :</p>
    <ol>
        <li>On initialise deux trajectoires très proches dans l'espace des phases, différenciées par un petit écart initial.</li>
        <li>On fait évoluer ces trajectoires en utilisant les équations du système dynamique.</li>
        <li>À chaque itération, on mesure la croissance de la divergence entre les trajectoires.</li>
        <li>En moyennant ces taux de divergence sur une certaine période, on obtient l'exposant de Lyapunov, qui caractérise la sensibilité aux conditions initiales du système.</li>
    </ol>
    <p>Cette quantification du chaos est essentielle pour comprendre la prédictibilité et la stabilité d'un système dynamique donné, ainsi que pour évaluer la validité de modèles théoriques et l'impact des erreurs expérimentales sur les résultats observés.</p>
    </div>
    <div style="flex: 1; padding-left: 20px; max-width: 50%;">
    <img src="images/explication lyapunov.png" width="400">
    </div>
</div>

### Analyse des résultats 

<img src="resultats/Exposant lyapunov.png" alt="Texte alternatif" style="height:700px;">


TODO :Cela rejoins le diagramme de bifurcation

<div style="display: flex;">
    <div style="flex: 1; padding-right: 20px; max-width: 50%;">
        <h2>Domaine non chaotique</h2>
        <p><p>Nous observons que les trajectoires des deux pendules sont confondues malgré des conditions initiales légèrement différentes. Cela indique un comportement non chaotique.</p>
</p>
        <p>Conditions initiales: $\theta_{10}$ = 10° $\theta_{20}$ = 20°</p>
        <video controls src="animations/lyapunov 1.mp4" style="width: 100%; max-width: 400px;" />
    </div>
    <div style="flex: 1; padding-left: 20px; max-width: 50%;">
        <h2>Domaine chaotique</h2>
        <p><p>Initialement, les trajectoires des deux pendules se superposent. Cependant, après un certain laps de temps, ces trajectoires divergent. Ce phénomène indique un comportement chaotique.</p>
</p>
        <p>Conditions initiales: $\theta_{10}$ = -60° $\theta_{20}$ = 85°</p>
        <video controls src="animations/lyapunov 2.mp4" style="width: 100%; max-width: 400px;" />
    </div>
</div>

### Conclusion : 



## 6.4 Critique des résultats et améliorations

<div style="display: flex;">
    <div style="flex: 1; padding-right: 20px; max-width: 50%;">
    <h2>Critiques</h2>
    <ul>
        <li>Optimisation de la méthode Newmark pour gagner en vitesse de calcul</li>
        <li>Améliorer la précision du calcul de l'exposant de Lyapunov pour que le résultat soit plus proche des valeurs obtenues avec le diagramme de bifurcation</li>
    </ul>
    </div>
    <div style="flex: 1; padding-left: 20px; max-width: 50%;">
    <h2>Améliorations</h2>
    <ul>
        <li>Détermination du temps de retournement du pendule double (lorsque l'extrémité du pendule se trouve plus haut que le point d'ancrage) en fonction des conditions initiales.</li>
        <li>Étude paramétrique en faisant varier les longueurs et masses du pendule double.</li>
    </ul>
    </div>
</div>